In [ ]:
%matplotlib inline


PyTorch: Control Flow + Weight Sharing
--------------------------------------

To showcase the power of PyTorch dynamic graphs, we will implement a very strange
model: a fully-connected ReLU network that on each forward pass randomly chooses
a number between 1 and 4 and has that many hidden layers, reusing the same
weights multiple times to compute the innermost hidden layers.



In [1]:
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

(0, 689.7920532226562)
(1, 684.3121337890625)
(2, 673.4315795898438)
(3, 682.7122802734375)
(4, 732.1351318359375)
(5, 638.2893676757812)
(6, 683.0774536132812)
(7, 678.3356323242188)
(8, 677.3494262695312)
(9, 676.2898559570312)
(10, 594.4865112304688)
(11, 677.0089111328125)
(12, 575.21533203125)
(13, 561.4002685546875)
(14, 543.1410522460938)
(15, 670.1819458007812)
(16, 668.3523559570312)
(17, 665.7644653320312)
(18, 474.66156005859375)
(19, 449.76434326171875)
(20, 654.01904296875)
(21, 384.1488037109375)
(22, 631.8275146484375)
(23, 616.6630859375)
(24, 350.31683349609375)
(25, 239.10899353027344)
(26, 544.433349609375)
(27, 511.49017333984375)
(28, 472.0292663574219)
(29, 430.403564453125)
(30, 469.1202697753906)
(31, 370.1134033203125)
(32, 409.05718994140625)
(33, 389.7032775878906)
(34, 376.1174011230469)
(35, 318.2449035644531)
(36, 215.1971435546875)
(37, 189.6103515625)
(38, 316.3470153808594)
(39, 215.4353485107422)
(40, 285.6966552734375)
(41, 204.41908264160156)
(42, 13

(415, 0.15116798877716064)
(416, 0.552038311958313)
(417, 0.7097259163856506)
(418, 0.10427719354629517)
(419, 0.3098117709159851)
(420, 0.16978657245635986)
(421, 1.1964861154556274)
(422, 0.24195222556591034)
(423, 0.8830258846282959)
(424, 0.6084325909614563)
(425, 0.5918800234794617)
(426, 0.9229487180709839)
(427, 0.7504206299781799)
(428, 0.4529557526111603)
(429, 0.13673575222492218)
(430, 0.5346847176551819)
(431, 0.14599613845348358)
(432, 0.4666462540626526)
(433, 0.3772630989551544)
(434, 0.15091900527477264)
(435, 0.2660301625728607)
(436, 0.24755322933197021)
(437, 0.756074845790863)
(438, 0.3098284602165222)
(439, 0.9939972758293152)
(440, 0.5796231627464294)
(441, 0.2565430700778961)
(442, 0.805530309677124)
(443, 0.4702143967151642)
(444, 0.126745343208313)
(445, 0.3743031919002533)
(446, 0.7716103196144104)
(447, 0.7008016109466553)
(448, 0.14494115114212036)
(449, 0.41058897972106934)
(450, 0.5601868033409119)
(451, 0.3778465688228607)
(452, 0.6257855296134949)
(453, 